## Data Preprocess

In [1]:
# Load libraries
import os
import re
import numpy as np
import pandas as pd
import requests
import pprint
import json
import urllib.request 
import bs4

from tqdm import tqdm
from lxml import html
from pandas.io.json import json_normalize

### 1. 메인 데이터(재비산먼지 농도) 수집 및 기본 처리

* API 인증키 에러가 지속되어 csv 파일로 하나씩 다운로드 받음(67개의 파일)
* 디렉토리 내의 파일들 모두 불러와 같은 형식으로 하나의 데이터프레임으로 합침
    * `date`에 연도가 붙지 않고 들어오는 데이터의 경우 임의로 붙여서 같은 형태 유지하도록 함
* '서울시'의 데이터만 활용 -> 메인 데이터프레임 별도 파일로 저장
* [링크](https://www.data.go.kr/data/15021888/fileData.do)

In [2]:
dirname = 'dust_data/'

In [4]:
# filenames = os.listdir(dirname)
# full_df = []
# # 컬럼의 순서는 같지만, 이름이 달라 모두 같은 형식으로 통일
# columns = ['date', 'time', 'city', 'gu', 'road', 'start', 'end', 'temperature', 'humidity', 'avg_re_dust', 'dust_ratio']
# for k, filename in enumerate(filenames):
#     full_filename = os.path.join(dirname, filename)
#     df = pd.read_csv(full_filename, encoding='cp949')
#     df.columns = columns
#     # `date` 형식이 다른 파일이 있어, 해당 파일은 전처리 후 합침
#     if len(df['date'][0]) != 10:
#         year = re.search(r"\d{4}", filename).group()
#         df['date'] = df['date'].apply(lambda x: year+'-'+x)
#     if k == 0:
#         # 첫 파일의 경우 합칠 데이터가 없어 그대로 데이터프레임 반환
#         full_df = df
#     else:
#         full_df = pd.concat([full_df, df], axis=0)

In [5]:
# 서울시 데이터만 활용할 예정
# seoul_df = full_df[full_df['city']=='서울']

In [6]:
# 별도의 파일로 저장
# seoul_df.to_csv('../data/seoul_re_dust.csv')

In [7]:
# seoul_df.head()

In [3]:
seoul_df = pd.read_csv('../data/seoul_re_dust.csv', index_col=0)
seoul_df['date_cd'] = seoul_df['date'].apply(lambda x: "".join(x.split('-')))

### 2. 서울시 생활인구 데이터 수집 및 처리

In [4]:
import dotenv

dotenv.load_dotenv()

True

In [27]:
# living_df = []
# for idx, date_cd in tqdm(enumerate(seoul_df['date_cd'].unique())):
#     living_api = f'http://openapi.seoul.go.kr:8088/{os.environ["living_api"]}/xml/SPOP_LOCAL_RESD_JACHI/1/600/{date_cd}'
#     response = requests.get(living_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 0:
#         living_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         living_df = pd.concat([living_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# living_df.to_csv('living_origin.csv')

In [28]:
# living_df = pd.read_csv('living_origin.csv')
# living_df['TOT_LVPOP_CO'] = living_df['TOT_LVPOP_CO'].astype(float)
# living_df_summ = living_df.groupby(['STDR_DE_ID', 'ADSTRD_CODE_SE'])['TOT_LVPOP_CO'].mean()
# living_df_new = living_df_summ.reset_index()
# living_df_new.to_csv('../data/living_df.csv')

In [29]:
# living_df = pd.read_csv('living_origin.csv')

### 3. 미세먼지

In [30]:
# fine_dust_df = []
# for idx, date_cd in tqdm(enumerate(seoul_df['date_cd'].unique())):
#     fine_dust_api = f'http://openapi.seoul.go.kr:8088/{os.environ["fine_dust_api"]}/xml/DailyAverageCityAir/1/24/{date_cd}'
#     response = requests.get(fine_dust_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 0:
#         fine_dust_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         fine_dust_df = pd.concat([fine_dust_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# fine_dust_df.to_csv('fine_dust_origin.csv')

In [31]:
# fine_dust_df.to_csv('../data/fine_dust_df.csv')

### 4. 건설현장

In [32]:
# contsruct_df = []

# for idx in range(1, 5000, 1000):
#     construct_api = f'http://openapi.seoul.go.kr:8088/{os.environ["construct_api"]}/xml/ListOnePMISBizInfo/{idx}/{idx+999}/'
#     response = requests.get(construct_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 1:
#         contsruct_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         contsruct_df = pd.concat([contsruct_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# contsruct_df.to_csv('construct_origin.csv')

In [13]:
# construct_df = contsrtuct_df[['DU_DATE', 'OFFICE_ADDR']]
# construct_df = construct_df[construct_df['OFFICE_ADDR'].apply(lambda x: True if "서울" in x else False)]

# construct_df['cons_start_dt'] = construct_df['DU_DATE'].apply(lambda x: x.split("~")[0])
# construct_df['cons_end_dt'] = construct_df['DU_DATE'].apply(lambda x: x.split("~")[1])
# construct_df['gu'] = construct_df['OFFICE_ADDR'].apply(lambda x: x.split(" ")[1])

# construct_df = construct_df.drop(['DU_DATE', 'OFFICE_ADDR'], axis=1)
# construct_df.to_csv('../data/contsruct_info.csv', index=False)

### 5. 버스 승하차 정보

In [53]:
# 행정동 버스 승하차 정보는 2022년 01월 28일부터 데이터가 제공됨
# seoul_df = seoul_df[seoul_df['date_cd']>='20220128']

In [48]:
# bus_df = []
# for idx, date_cd in tqdm(enumerate(seoul_df['date_cd'].unique())):
#     living_api = f'http://openapi.seoul.go.kr:8088/{os.environ["bus_api"]}/xml/tpssEmdBus/1/424/{date_cd}'
#     response = requests.get(living_api)
#     contents = response.text

#     xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
#     rows = xml_obj.findAll('row')
#     # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
#     row_list = [] # 행값
#     name_list = [] # 열이름값
#     value_list = [] #데이터값

#     # xml 안의 데이터 수집
#     for i in range(0, len(rows)):
#         columns = rows[i].find_all()
#         #첫째 행 데이터 수집
#         for j in range(0,len(columns)):
#             if i ==0:
#                 # 컬럼 이름 값 저장
#                 name_list.append(columns[j].name)
#             # 컬럼의 각 데이터 값 저장
#             value_list.append(columns[j].text)
#         # 각 행의 value값 전체 저장
#         row_list.append(value_list)
#         # 데이터 리스트 값 초기화
#         value_list=[]

#     if idx == 0:
#         bus_df = pd.DataFrame(row_list, columns=name_list)
#     else:
#         bus_df = pd.concat([bus_df, pd.DataFrame(row_list, columns=name_list)], axis=0)

# bus_df.to_csv('bus_origin.csv')

141it [01:20,  1.76it/s]


In [5]:
# bus_df.to_csv('../data/bus_df.csv')